In [ ]:
%load_ext autoreload
%autoreload 2

import os
import time
import cv2
import torch
import numpy as np

In [ ]:
from scripts.utils import np_to_tesor_img, tensor_to_np_img, save_tensor_img, open_img_as_tensor, display_img

## Face Detector Models

In [ ]:
from scripts.face_detectors import MediaPipe, YuNet, YoloFace

In [ ]:
mp = MediaPipe()
yn = YuNet()
yf = YoloFace(conf=0.9)

## Our Model

In [ ]:
from scripts.iarm import *
iarm = IARM()

## Image Feature Extraction

In [ ]:
from scripts import image_attributes

In [ ]:
# Settings
image_attributes.save_color_images = False
image_attributes.save_lbp_images = False
image_attributes.save_gradient_images = False

## FGSM

Model for feature extraction

In [ ]:
print("CUDA Available: ", torch.cuda.is_available())
main_yf = YoloFace()
device, model = main_yf.device, main_yf.yf_face_detector
model.eval()

In [ ]:
from scripts import fgsm

Sample attack without mask

In [ ]:
input_file = r"./input/sample1.jpg"

In [ ]:
t1 = time.time()

input_img = open_img_as_tensor(input_file)
feats, grads, bboxes, masks = image_attributes.get_features(input_file)
preds = iarm.predict(feats)
output_img = fgsm.fgsm_attack(input_img, preds, grads, masks)

t2 = time.time()

print("Our regression model run time:", t2 - t1)
display_img(output_img)

Sample attack with mask

In [ ]:
input_file = r"./input/sample2.jpg"

In [ ]:
from scripts.facesegmentor import FaceSegementor
faceseg = FaceSegementor()

t1 = time.time()

input_img = open_img_as_tensor(input_file)
feats, grads, bboxes, masks = image_attributes.get_features(input_file, face_segmentor=faceseg)
preds = iarm.predict(feats)
output_img = fgsm.fgsm_attack(input_img, preds, grads, masks)

t2 = time.time()

print("Our regression model run time:", t2 - t1)
display_img(output_img)

Compare with binary search

In [ ]:
input_file = r"./input/sample3.jpg"

In [ ]:
t1 = time.time()

input_img = open_img_as_tensor(input_file)
_, grads, bboxes, masks = image_attributes.get_features(input_file)
e_mins = []

for data_grad, mask, bbox in zip(grads, masks, bboxes):
    e_mins.append(fgsm.binary_search(input_img, data_grad, yf, mask, bbox))

output_img = fgsm.fgsm_attack(input_img, e_mins, grads, masks)

t2 = time.time()

print("Our regression model run time:", t2 - t1)
display_img(output_img)

Check if the faces can still be detected

In [ ]:
input_file = r"./input/sample1.jpg"

In [ ]:
input_img = open_img_as_tensor(input_file)
feats, grads, bboxes, masks = image_attributes.get_features(input_file)
preds = iarm.predict(feats, multiplier=2.5)
output_img = fgsm.fgsm_attack(input_img, preds, grads, masks)

yf.detect("Detected faces:", output_img)

In [ ]:
input_file = r"./input/sample2.jpg"

In [ ]:
input_img = open_img_as_tensor(input_file)
feats, grads, bboxes, masks = image_attributes.get_features(input_file)
preds = iarm.predict(feats, multiplier=2.5)
output_img = fgsm.fgsm_attack(input_img, preds, grads, masks)

yf.detect("Detected faces:", output_img)

##### ___